# File Name: simple_sagemaker_bedrock.ipynb
### Location: Chapter 3
### Purpose: 
#####             1. Set up an Amazon Bedrock guardrail using the API.
#####             2. Test and monitor the guardrail during prompt interactions.
#####             3. Test and monitor the guardrail during response handling.
#####             4. Execute and monitor the complete guardrail policy.
##### Dependency: Not Applicable
# <ins>-----------------------------------------------------------------------------------</ins>

# <ins>Amazon SageMaker Classic</ins>
#### Those who are new to Amazon SageMaker Classic. Follow the link for the details. https://docs.aws.amazon.com/sagemaker/latest/dg/studio.html

# <ins>Environment setup of Kernel</ins>
##### Fill "Image" as "Data Science"
##### Fill "Kernel" as "Python 3"
##### Fill "Instance type" as "ml-t3-medium"
##### Fill "Start-up script" as "No Scripts"
##### Click "Select"

###### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/notebooks-create-open.html for details.

# <ins>Mandatory installation on the kernel through pip</ins>

##### This lab will work with below software version. But, if you are trying with latest version of boto3, awscli, and botocore. This code may fail. You might need to change the corresponding api. 

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell. 

In [ ]:
%pip install --no-build-isolation --force-reinstall -q \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

# <ins>Disclaimer</ins>

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell.

# <ins>Restart the kernel</ins>

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

# <ins>Python package import</ins>

##### boto3 offers various clients for Amazon Bedrock to execute various actions.
##### botocore is a low-level interface to AWS tools, while boto3 is built on top of botocore and provides additional features

In [ ]:
import json
import os
import botocore
import boto3
import warnings
import time
import random
import sys
import base64
import io
from PIL import Image
from IPython.display import clear_output, display, display_markdown, Markdown

### Ignore warning 

In [ ]:
warnings.filterwarnings('ignore')

# <ins>Amazon Bedrock Runtime Client</ins>

##### Purpose: used for making inference requests for models hosted in Amazon Bedrock. 
##### Refer https://docs.aws.amazon.com/bedrock/latest/APIReference/API_Operations_Amazon_Bedrock_Runtime.html for details about Amazon Bedrock runtime client 

## Define important environment variable

# <ins>Amazon Bedrock Client</ins>

##### Purpose: used for managing, training, and deploying models on Amazon Bedrock
##### Refer https://docs.aws.amazon.com/bedrock/latest/APIReference/API_Operations_Amazon_Bedrock.html for details about Amazon Bedrock client 

In [ ]:
# Try-except block to handle potential errors
try:
    # Create a new Boto3 session to interact with AWS services
    boto3_session_name = boto3.session.Session()

    # Retrieve the current AWS region from the session
    aws_region_name = boto3_session_name.region_name
    
    # Create a new Boto3 bedrock client to interact with AWS services
    boto3_bedrock_client = boto3.client('bedrock')
    
    # Create a new Boto3 bedrock runtime client to interact with AWS services
    boto3_bedrock_runtime_client = boto3.client('bedrock-runtime', region_name = aws_region_name,)
    
    # Generate a random suffix number between 200 and 900
    random_suffix = random.randrange(200, 900)
    
    # Store all variables in a dictionary
    variables_store = {
        "boto3_session_name": boto3_session_name,
        "aws_region_name": aws_region_name,
        "boto3_bedrock_client": boto3_bedrock_client,
        "random_suffix": random_suffix,
        "boto3_bedrock_runtime_client": boto3_bedrock_runtime_client
    }

    # Print all variables
    for var_name, value in variables_store.items():
        print(f"{var_name}: {value}")

except Exception as e:
    print(f"An unexpected error occurred: {e}")


# <ins>Amazon Bedrock Runtime Client</ins>

##### Purpose: used for making inference requests for models hosted in Amazon Bedrock. 
##### Refer https://docs.aws.amazon.com/bedrock/latest/APIReference/API_Operations_Amazon_Bedrock_Runtime.html for details about Amazon Bedrock runtime client 

# <ins>Find out list of foundation models on Amazon Bedrock</ins>

##### API name: list_foundation_models 
##### Documentation: https://docs.aws.amazon.com/bedrock/latest/APIReference/API_ListFoundationModels.html
##### You can find out both foundation and customized model on Amazon Bedrock

In [ ]:
try:
    # Retrieve the list of foundation models available in Bedrock
    model_summaries = boto3_bedrock_client.list_foundation_models().get('modelSummaries', [])
    
    # Check if model summaries are available and print them
    if model_summaries:
        print("Available Foundation Models:")
        for model in model_summaries:
            print(f"Model ID: {model.get('modelId')}, Model Name: {model.get('modelName')}")
    else:
        print("No foundation models found.")
    
except botocore.exceptions.ClientError as error:
    # Handle errors if there is an issue with the request
    error_code = error.response['Error'].get('Code', 'Unknown')
    print(f"An error occurred: {error_code}")
    if error_code == 'AccessDeniedException':
        print("Access Denied: Please check your permissions for Bedrock services.")
    else:
        print("An unexpected error occurred.")
        raise  # Reraise other exceptions for further debugging if needed


# <ins>Example of Amazon Titan LLM foundation model</ins>

##### This example is based on Titan Text G1 - Express v1 foundation model. 
##### Model ID: amazon.titan-text-express-v1
##### Refer https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-titan-text.html

##### API: invoke_model
##### Refer https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModel.html

# <ins>Disclaimer</ins> 

##### Make sure that amazon.titan-text-express-v1 is allowlisted on Amazon Bedrock model access. Refer Section 3.3 of Chapter 3 of the Book

In [ ]:
## Defining model_id, prompt and other variables
## You can try out different model id, your own prompt. 
model_id = "amazon.titan-text-express-v1"
prompt = """User: Generate a story for a kid about beauty of a rainbow within 100 words
bot:
"""

##### <ins>Example with default inferance parameters</ins>

In [ ]:
try:
    # Prepare the input payload for the model invocation
    body = json.dumps({"inputText": prompt})
    modelId = model_id
    accept = "application/json"
    contentType = "application/json"

    # Invoke the model with the specified parameters using boto3 client
    response = boto3_bedrock_runtime_client.invoke_model(
        body=body,
        modelId=modelId,
        accept=accept,
        contentType=contentType
    )
    
    # Parse the response body to extract the model's output
    response_body = json.loads(response.get("body").read())
    output_text = response_body.get("results", [{}])[0].get("outputText", "No output text available")

    # Display the output text from the model
    print("Model Output:", output_text)

except botocore.exceptions.ClientError as error:
    # Handle client errors and identify specific error types
    error_code = error.response['Error'].get('Code', 'Unknown')
    
    # Provide a specific message if access is denied
    if error_code == 'AccessDeniedException':
        print(f"\x1b[41mAccess Denied: {error.response['Error'].get('Message', 'No message available')}\x1b[0m")
    else:
        # Print a generic error message for other issues
        print(f"An error occurred: {error}")
        raise  # Re-raise the exception for further handling if needed


##### <ins>Example with  inferance parameters configuration</ins>

In [ ]:
try:
    # Construct the request payload with prompt and generation configuration
    body = json.dumps({
        "inputText": prompt,
        "textGenerationConfig": {
            "topP": 0.95,          # Controls the nucleus sampling probability (diversity of output)
            "temperature": 0.2     # Controls the creativity of the model's response
        }
    })

    # Define model and content parameters
    modelId = model_id
    accept = "application/json"
    contentType = "application/json"

    # Invoke the model with the boto3 Bedrock client
    response = boto3_bedrock_runtime_client.invoke_model(
        body=body,
        modelId=modelId,
        accept=accept,
        contentType=contentType
    )
    
    # Parse the response body to retrieve the model output
    response_body = json.loads(response.get("body").read())
    output_text = response_body.get("results", [{}])[0].get("outputText", "No output text available")

    # Print the generated output text
    print("Model Output:", output_text)

except botocore.exceptions.ClientError as error:
    # Handle errors related to the ClientError exception
    error_code = error.response['Error'].get('Code', 'Unknown')
    
    # Specific handling for Access Denied errors with a styled output
    if error_code == 'AccessDeniedException':
        print(f"\x1b[41mAccess Denied: {error.response['Error'].get('Message', 'No message available')}\x1b[0m")
    else:
        # Print a generic message for other ClientErrors and re-raise the error
        print(f"An error occurred: {error}")
        raise  # Reraise to allow further handling or debugging if necessary


# <ins>Example of Anthropic LLM foundation model</ins>

##### This example is based on Claude 3 Haiku foundation model. 
##### Model ID: anthropic.claude-3-haiku-20240307-v1:0
##### Refer https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html

##### API: invoke_model
##### Refer https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModel.html

# <ins>Disclaimer</ins> 

##### Make sure that anthropic.claude-3-haiku-20240307-v1:0 is allowlisted on Amazon Bedrock model access. Refer Section 3.3 of Chapter 3 of the Book

In [ ]:
## Defining model_id, prompt and other variables
## You can try out different model id, your own prompt. 
model_id = "anthropic.claude-3-haiku-20240307-v1:0"
prompt = """Human: Generate a story for a kid about beauty of a rainbow within 100 words

Assitance:
"""

##### <ins>Example with default inferance parameters</ins>

In [ ]:
try:
    # Construct the request payload with message info and model configuration
    messages_info = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,  # Limit the response tokens to control output length
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt  # User-provided prompt for the model
                    }
                ]
            }
        ]
    }
    
    # Convert the payload to JSON format
    body = json.dumps(messages_info)
    
    # Define model parameters and content type
    modelId = model_id
    accept = "application/json"
    contentType = "application/json"

    # Invoke the model using the boto3 Bedrock runtime client
    response = boto3_bedrock_runtime_client.invoke_model(
        body=body,
        modelId=model_id,
        accept=accept,
        contentType=contentType
    )

    # Parse the response body to extract the model's response content
    response_body = json.loads(response.get("body").read())
    output_text = response_body.get("content", [{}])

    # Print the first part of the text response if available
    if output_text and isinstance(output_text, list) and 'text' in output_text[0]:
        print("Model Output:", output_text[0]['text'])
    else:
        print("No valid text response received from the model.")

except botocore.exceptions.ClientError as error:
    # Handle exceptions specifically for ClientError
    error_code = error.response['Error'].get('Code', 'Unknown')
    
    # Specific handling for Access Denied errors
    if error_code == 'AccessDeniedException':
        print(f"\x1b[41mAccess Denied: {error.response['Error'].get('Message', 'No message available')}\x1b[0m")
    else:
        # Print a generic message for other ClientErrors
        print(f"An error occurred: {error}")
        raise  # Reraise the error for further handling or debugging if necessary


##### <ins>Example with  inferance parameters configuration</ins>

In [ ]:
import json
import botocore.exceptions

try:
    # Define the message payload with model configuration for generation
    messages_info = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,         # Maximum number of tokens in the response
        "temperature": 0.9,         # Controls response creativity; higher values increase randomness
        "top_p": 0.8,               # Nucleus sampling probability for diverse responses
        "top_k": 20,                # Limits the number of words to sample from at each step
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt  # User-provided prompt for the model
                    }
                ]
            }
        ]
    }

    # Convert payload to JSON format for model invocation
    body = json.dumps(messages_info)
    
    # Define model parameters and headers
    modelId = model_id
    accept = "application/json"
    contentType = "application/json"

    # Call the Bedrock model using the boto3 runtime client
    response = boto3_bedrock_runtime_client.invoke_model(
        body=body,
        modelId=model_id,
        accept=accept,
        contentType=contentType
    )

    # Parse the JSON response to retrieve the model output
    response_body = json.loads(response.get("body").read())
    output_text = response_body.get("content", [{}])

    # Print the response text if it's available and valid
    if output_text and isinstance(output_text, list) and 'text' in output_text[0]:
        print("Model Output:", output_text[0]['text'])
    else:
        print("No valid text response received from the model.")

except botocore.exceptions.ClientError as error:
    # Error handling for ClientError, checking the specific error code
    error_code = error.response['Error'].get('Code', 'Unknown')
    
    # Specific handling if access is denied, with styled error output
    if error_code == 'AccessDeniedException':
        print(f"\x1b[41mAccess Denied: {error.response['Error'].get('Message', 'No message available')}\x1b[0m")
    else:
        # Generic error message for other ClientErrors
        print(f"An error occurred: {error}")
        raise  # Re-raise the error for further handling if necessary


# <ins>Example of Amazon Titan Image foundation model</ins>

##### This example is based on Amazon Titan image generator version 2 foundation model. 
##### Model ID: amazon.titan-image-generator-v2:0
##### Refer https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-titan-image.html

##### API: invoke_model
##### Refer https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModel.html

# <ins>Disclaimer</ins> 

##### Make sure that amazon.titan-image-generator-v2:0 is allowlisted on Amazon Bedrock model access. Refer Section 3.3 of Chapter 3 of the Book

In [ ]:
## Defining model_id, prompt and other variables
## You can try out different model id, your own prompt. 
model_id = "amazon.titan-image-generator-v2:0"
prompt = """An image of a cat with a big hat at a park
"""

##### <ins>Example with default inferance parameters</ins>

In [ ]:
try:
    # Define the request payload for text-to-image generation
    body = json.dumps({
        "taskType": "TEXT_IMAGE",            # Task type indicating text-to-image generation
        "textToImageParams": {
            "text": prompt                   # User-provided prompt for image generation
        }
    })
    
    # Define model parameters and headers
    modelId = model_id
    accept = "application/json"
    contentType = "application/json"

    # Invoke the model using the boto3 Bedrock runtime client
    response = boto3_bedrock_runtime_client.invoke_model(
        body=body,
        modelId=model_id,
        accept=accept,
        contentType=contentType
    )

    # Parse the JSON response to retrieve model's response content
    response_body = json.loads(response.get("body").read())

    # Check if the response contains an image or relevant information
    image_data = response_body.get("image", None)
    if image_data:
        print("Generated Image Data:", image_data)
    else:
        print("No image data received in the response.")

except botocore.exceptions.ClientError as error:
    # Error handling for ClientError, with a check for access denial
    error_code = error.response['Error'].get('Code', 'Unknown')
    
    if error_code == 'AccessDeniedException':
        # Highlight access denied error with a colored message
        print(f"\x1b[41mAccess Denied: {error.response['Error'].get('Message', 'No message available')}\x1b[0m")
    else:
        # Generic error message for other ClientErrors
        print(f"An error occurred: {error}")
        raise  # Re-raise the error for further handling if necessary


#### <ins>The output of invoke model is a base64 encoded string of the image data. You need to convert this with below code to see the image.</ins>

In [ ]:
try:
    # Get the base64 encoded image from the response body
    base64_image = response_body.get("images")[0]
    
    if base64_image:
        # Decode the base64 string into bytes
        base64_bytes = base64_image.encode('ascii')
        image_bytes = base64.b64decode(base64_bytes)
        
        # Open the image using PIL
        image = Image.open(io.BytesIO(image_bytes))
        
        # Display the image
        display(image)
    else:
        print("No image data found in the response.")
        
except Exception as e:
    print(f"An error occurred while processing the image: {e}")


##### <ins>Example with  inferance parameters configuration</ins>

In [ ]:
import json
import botocore.exceptions
import base64
import io
from PIL import Image
from IPython.display import display

# Define the number of images you want to generate
numberOfImages = 3

try:
    # Prepare the request payload for text-to-image generation
    body = json.dumps({
        "taskType": "TEXT_IMAGE",  # Task type indicating text-to-image generation
        "textToImageParams": {
            "text": prompt  # User-provided prompt for image generation
        },
        "imageGenerationConfig": {
            "numberOfImages": numberOfImages,  # Specify number of images to generate
            "height": 1024,  # Height of the generated images
            "width": 1024,   # Width of the generated images
            "cfgScale": 8.0,  # CFG Scale for image generation
            "seed": 0         # Seed value for randomization
        }
    })
    
    # Set headers and model ID
    modelId = model_id
    accept = "application/json"
    contentType = "application/json"

    # Call the model to generate images
    response = boto3_bedrock_runtime_client.invoke_model(
        body=body, modelId=model_id, accept=accept, contentType=contentType
    )

    # Parse the JSON response
    response_body = json.loads(response.get("body").read())

    # Check if the response contains images
    images = response_body.get("images", [])
    if images:
        for idx, base64_image in enumerate(images[:numberOfImages]):
            # Decode the base64 string to bytes
            base64_bytes = base64_image.encode('ascii')
            image_bytes = base64.b64decode(base64_bytes)
            
            # Open and display the image using PIL
            image = Image.open(io.BytesIO(image_bytes))
            print(f"Displaying Image {idx + 1}")
            display(image)
    else:
        print("No image data found in the response.")

except botocore.exceptions.ClientError as error:
    # Handle client errors, specifically Access Denied
    error_code = error.response['Error'].get('Code', 'Unknown')
    if error_code == 'AccessDeniedException':
        print(f"\x1b[41mAccess Denied: {error.response['Error'].get('Message', 'No message available')}\x1b[0m")
    else:
        print(f"An error occurred: {error}")
        raise  # Re-raise the error for further handling
except Exception as e:
    # Handle any other exceptions
    print(f"An unexpected error occurred: {e}")


# <ins>Example of Amazon Titan LLM foundation model with streaming API</ins>


##### This example is based on Titan Text G1 - Express v1 foundation model. 
##### Model ID: amazon.titan-text-express-v1
##### Refer https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-titan-text.html

##### API: invoke_model_with_response_stream
##### Refer https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModelWithResponseStream.html

# <ins>Disclaimer</ins> 

##### Make sure that amazon.titan-text-express-v1 is allowlisted on Amazon Bedrock model access. Refer Section 3.3 of Chapter 3 of the Book

In [ ]:
## Defining model_id, prompt and other variables
## You can try out different model id, your own prompt. 
model_id = "amazon.titan-text-express-v1"
prompt = """User: Generate a story for a kid about beauty of a rainbow within 1000 words
bot:
"""

##### <ins>Example with default inferance parameters</ins>

In [ ]:
try:
    # Prepare the body for the request
    body = json.dumps({"inputText": prompt})
    modelId = model_id
    accept = "application/json"
    contentType = "application/json"

    # Call the invoke_model_with_response_stream API to get the response
    response = boto3_bedrock_runtime_client.invoke_model_with_response_stream(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    
    # Get the response body stream
    stream = response.get('body')

    if stream:
        # Process each event in the stream
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                # Decode and load the JSON chunk
                chunk_obj = json.loads(chunk.get('bytes').decode())
                
                # Extract the output text from the chunk
                text = chunk_obj.get('outputText')
                if text:
                    # Clear the previous output to avoid cluttering
                    clear_output(wait=True)
                    
                    # Display the output text in markdown format
                    display_markdown(Markdown(text))
                else:
                    print("No output text found in this chunk.")
    else:
        print("No data stream available.")

except botocore.exceptions.ClientError as error:
    # Handle AWS client errors
    error_code = error.response['Error'].get('Code', 'Unknown')
    if error_code == 'AccessDeniedException':
        print(f"\x1b[41mAccess Denied: {error.response['Error'].get('Message', 'No message available')}")
    else:
        print(f"An error occurred: {error}")
        raise


##### <ins>Example with  inferance parameters configuration</ins>

In [ ]:
try:
    # Prepare the request body with the desired configurations
    body = json.dumps({
        "inputText": prompt,
        "textGenerationConfig": {
            "topP": 0.95,
            "temperature": 0.2
        }
    })
    
    modelId = model_id
    accept = "application/json"
    contentType = "application/json"

    # Call the invoke_model_with_response_stream API to get the response stream
    response = boto3_bedrock_runtime_client.invoke_model_with_response_stream(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    
    # Get the response body stream
    stream = response.get('body')

    if stream:
        # Process each event in the stream
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                # Decode and load the JSON chunk
                chunk_obj = json.loads(chunk.get('bytes').decode())
                
                # Extract and print the output text
                text = chunk_obj.get('outputText')
                if text:
                    clear_output(wait=True)  # Clear the previous output
                    display_markdown(Markdown(text))  # Display the output in markdown format
    else:
        print("No data stream available.")

except botocore.exceptions.ClientError as error:
    # Handle AWS client errors
    error_code = error.response['Error'].get('Code', 'Unknown')
    if error_code == 'AccessDeniedException':
        print(f"\x1b[41mAccess Denied: {error.response['Error'].get('Message', 'No message available')}")
    else:
        print(f"An error occurred: {error}")
        raise


# End of NoteBook 

## Please ensure that you close the kernel after using this notebook to avoid any potential charges to your account.

## Process: Go to "Kernel" at top option. Choose "Shut Down Kernel". 
##### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/studio-ui.html